In [1]:
from trading import Environment
from trading.strategies import Sma2Porcentaje
from trading.adapter import Data2SmaPorcentaje
from datetime import datetime
from backtesting import Backtest
from TradingFunctions import Strategy2sma_porcentaje
import pandas as pd

In [26]:
adapter = Data2SmaPorcentaje(n_sma1= 91, n_sma2= 40)
strategy = Sma2Porcentaje(factor_sll= 0.903, facto_sls= 1.022, factor_swl= 1.095, factor_sws= 0.9)

In [27]:
env = Environment('dataset.csv', data_adapter= adapter, cash= 1)

In [22]:
env.actual_state()

open_time          1.577864e+12
open               7.221073e+03
high               7.221073e+03
low                7.218054e+03
close              7.218054e+03
sma_91             7.217510e+03
sma_40             7.224847e+03
sma_91_prev        7.217166e+03
sma_40_prev        7.224916e+03
reference_point    0.000000e+00
state              2.000000e+00
dtype: float64

In [28]:
state = env.actual_state()
while True:
    action = strategy.next(state)
    reward, state, end = env.step(action)

    if end: break

In [29]:
env.get_summary()

Return [%]                                    619.1


In [30]:
env.get_money()

6.200647468921302

In [31]:
data = pd.read_csv("dataset.csv")
data = data.rename({'open':'Open',
                    'high':'High',
                    'low':'Low',
                    'close':'Close'},axis=1).drop(['volume'],axis=1)
data = data[data['open_time']%(5*60000)==0]
data['open_time'] = pd.DatetimeIndex( data['open_time'].apply(lambda t:datetime.fromtimestamp(t/1000)) )
data = data.set_index('open_time')
data = data.sort_index(ascending=True)

In [32]:
bt = Backtest(data, Strategy2sma_porcentaje, cash= 1e6, commission= .0005, exclusive_orders=True)

In [33]:
bt.run(x = [91, 40, 1.095, 0.903, 0.9, 1.022])

Start                     2019-12-31 21:00:00
End                       2024-02-29 20:55:00
Duration                   1520 days 23:55:00
Exposure Time [%]                   96.807427
Equity Final [$]               6892830.126998
Equity Peak [$]                8337505.021478
Return [%]                         589.283013
Buy & Hold Return [%]              753.257178
Return (Ann.) [%]                   58.877054
Volatility (Ann.) [%]              115.643898
Sharpe Ratio                         0.509124
Sortino Ratio                        1.307337
Calmar Ratio                         0.753252
Max. Drawdown [%]                  -78.163832
Avg. Drawdown [%]                   -1.612324
Max. Drawdown Duration     1051 days 18:45:00
Avg. Drawdown Duration        3 days 02:49:00
# Trades                                  153
Win Rate [%]                        65.359477
Best Trade [%]                      12.998571
Worst Trade [%]                    -21.971497
Avg. Trade [%]                    